<a href="https://colab.research.google.com/github/Flychuban/Stocks-Crypto-Research/blob/main/Stocks_Crypto_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [3]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) 

In [ ]:
url = "https://uk.finance.yahoo.com/news/d-put-2-000-tesla-043029225.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [ ]:
ARTICLE

'RCB’s owners have focused on Virat Kohli to leverage their commercial brand Both Tesla (NASDAQ: TSLA) and NIO (NYSE: NIO) stock declined by more than 50% in value in 2022. It was a dreadful year for most growth shares, including electric vehicle (EV) companies. But what if I’d taken a contrarian stance and decided to invest £1,000 in each of these fallen stocks as a New Year gift for myself? How much would I have today? Well, Tesla shares are up a very impressive 65% so far this year. In contrast, NIO shares have declined 17% since the end of December and now sit at just under $8 per share. This means that my Tesla holding would be worth £1,650, while the value of my position in its Chinese EV rival would have fallen to £830. So, my overall investment would be worth £2,480 today. That’s a gain of 24%, which is an exceptional return after just a few months. But what about the future? Should I buy either or both stocks today? There seem to be two big reasons why Tesla stock has come bac

In [ ]:
input_ids = tokenizer.encode(ARTICLE, return_tensors = 'pt')
output = model.generate(input_ids, max_length = 55, num_beams = 5, early_stopping = True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
summary

'Tesla stock is up 65% so far this year, while NIO shares are down 17%.'

In [20]:
monitored_tickers = ['TMUS', 'RCL', 'ETH']

In [21]:
def search_stock_news_urls(ticker):
  search_url = f'https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws'
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

In [ ]:
search_stock_news_urls('TSLA')

['https://accounts.google.com/ServiceLogin?hl=nl&continue=https://www.google.com/search?q%3Dyahoo%2Bfinance%2BTSLA%26tbm%3Dnws&gae=cb-none',
 'https://policies.google.com/technologies/cookies?hl=nl&utm_source=ucb',
 'https://consent.google.com/dl?continue=https://www.google.com/search?q%3Dyahoo%2Bfinance%2BTSLA%26tbm%3Dnws&gl=NL&hl=nl&pc=srp&uxe=none&src=1',
 'https://policies.google.com/privacy?hl=nl&utm_source=ucb',
 'https://policies.google.com/terms?hl=nl&utm_source=ucb']

In [22]:
raw_urls = {ticker: search_stock_news_urls(ticker) for ticker in monitored_tickers}

In [23]:
raw_urls

{'TMUS': ['/?sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQOwgC',
  '/search?q=yahoo+finance+TMUS&tbm=nws&ie=UTF-8&gbv=1&sei=vqtrZPmVOaak2roP8N-N4A4',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUIBSgA',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUIBygC',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+TMUS&um=1&ie=UTF-8&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUICSgE',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUICigF',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwj57Ijrvon_AhUmklYBHfBvA-wQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+TMUS&ie=UTF-8&tbm=nws&source=lnt&tbs=lr:lang_1zh-CN%7Clang_1zh-TW&lr=lang_zh-CN%7Clang_zh-TW&sa=X&ved=0ahUKEwj57Ijrvon_A

In [24]:
import re

In [25]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [26]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [27]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'TMUS': ['https://ca.finance.yahoo.com/news/corporate-board-veteran-netflix-pivoted-successfully-on-social-and-political-issues-140106210.html',
  'https://finance.yahoo.com/news/top-research-reports-mastercard-coca-164804069.html',
  'https://finance.yahoo.com/news/t-mobile-delivers-industry-leading-200200200.html',
  'https://finance.yahoo.com/news/t-mobile-celebrates-small-business-130900749.html',
  'https://finance.yahoo.com/news/investors-more-bullish-t-mobile-110031458.html',
  'https://finance.yahoo.com/news/t-mobile-tmus-stock-moves-220010714.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BTMUS%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/3-wireless-stocks-likely-thrive-135301120.html',
  'https://finance.yahoo.com/news/t-mobile-us-nasdaq-tmus-120018259.html',
  'https://finance.yahoo.com/news/biden-official-jury-is-still-out-on-how-6g-will-look-173257234.html',
  'https://finance.yahoo.com/news/feeding-speed-ts-grand-42-150928145.html'],
 'RCL': [

In [28]:
def scrape_and_process(URLs):
  ARTICLES = []
  for url in URLs:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = [paragraph.text for paragraph in paragraphs]
    words = ' '.join(text).split(' ')[:250]
    ARTICLE = ' '.join(words)
    ARTICLES.append(ARTICLE)
  return ARTICLES

In [29]:
articles = {ticker: scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'TMUS': ['As election season approaches, companies must increasingly think about how they approach the social and political issues that will undoubtedly resurface. "I think it\'s very important that companies reflect on what the core of their business is," Betsy Atkins, chair of Alphabet\'s (GOOG, GOOGL) Google Cloud and board member at Wynn and GoPuff, told Yahoo Finance at the 2023 Milken Global Conference (video above). "Is it building a fantastic product, a fantastic service, or is it about trying to reshape the world? When companies decide that social and political issues are their most important thing, that\'s when they get into trouble." Atkins floated Netflix (NFLX) as an example. "They had such a unified view and were on one side of the political spectrum that their creative teams created content for only one side," Atkins said. "Finally, they realized they were losing half the market. So they had to go back and say, at Netflix, we\'re going to create content for all." On the

In [30]:
def summarize(articles):
  summaries = []
  for article in articles:
    input_ids = tokenizer.encode(article, return_tensors = 'pt')
    output = model.generate(input_ids, max_length = 55, num_beams = 5, early_stopping = True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    summaries.append(summary)
  return summaries

In [31]:
summaries = {ticker: summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'TMUS': ['Atkins says companies get into trouble if they focus too much on politics.',
  'Shares of Mastercard have outperformed the Financial Transaction Services industry over the past year.',
  'Industry-Leading Postpaid customer additions of 287 thousand, more than AT&T, Verizon and Charter. T-Mobile to buy MetroPCS in $4.8 billion deal',
  'T-Mobile Better Choice Bundle starts at $65 a month with no annual contracts, leasing or installation fees. Small businesses can save over 30% compared to Big Cable',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.',
  ',,,,,,',
  'Carriers face challenges from soft China market, margin erosion and supply-chain disruptions.',
  'T-Mobile US has increased its earnings per share (EPS) by 9.8% over five years.',
  'Top Biden officials gather at National Science Foundation. Experts warn that changes could be felt as early as this decade',
  'Telecom giant plans to spend $42.

In [54]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [56]:
MODEL_NAME = 'RashidNLP/Finance_Multi_Sentiment'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 3).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [55]:
def get_sentiment(sentences):
    bert_dict = {}
    vectors = tokenizer(sentences, padding = True, max_length = 65, return_tensors='pt').to(device)
    outputs = bert_model(**vectors).logits
    probs = torch.nn.functional.softmax(outputs, dim = 1)
    for prob in probs:
        bert_dict['neg'] = round(prob[0].item(), 3)
        bert_dict['neu'] = round(prob[1].item(), 3)
        bert_dict['pos'] = round(prob[2].item(), 3)
        print (bert_dict)

In [65]:
scores = {ticker: get_sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'neg': 0.004, 'neu': 0.002, 'pos': 0.994}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0}
{'neg': 0.0, 'neu': 0.002, 'pos': 0.998}
{'neg': 0.005, 'neu': 0.919, 'pos': 0.076}
{'neg': 0.005, 'neu': 0.919, 'pos': 0.076}
{'neg': 0.006, 'neu': 0.256, 'pos': 0.738}
{'neg': 0.699, 'neu': 0.025, 'pos': 0.275}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0}
{'neg': 0.168, 'neu': 0.021, 'pos': 0.811}
{'neg': 0.0, 'neu': 0.004, 'pos': 0.996}
{'neg': 0.0, 'neu': 0.001, 'pos': 0.999}
{'neg': 0.0, 'neu': 0.02, 'pos': 0.979}
{'neg': 0.005, 'neu': 0.919, 'pos': 0.076}
{'neg': 0.006, 'neu': 0.256, 'pos': 0.738}
{'neg': 0.005, 'neu': 0.919, 'pos': 0.076}
{'neg': 0.0, 'neu': 0.002, 'pos': 0.998}
{'neg': 0.005, 'neu': 0.919, 'pos': 0.076}
{'neg': 0.0, 'neu': 0.003, 'pos': 0.997}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0}
{'neg': 0.007, 'neu': 0.432, 'pos': 0.561}
{'neg': 0.0, 'neu': 0.001, 'pos': 0.999}
{'neg': 0.003, 'neu': 0.066, 'pos': 0.932}
{'neg': 0.0, 

{'TMUS': None, 'RCL': None, 'ETH': None}